In [1]:
import cntk as C
from cntk.device import try_set_default_device, gpu
try_set_default_device(gpu(0))

import numpy as np
import time

from torch.utils.data import DataLoader

# Hessian Vector Product (HVP)

Implement HVP with numerical gradient method.

i.e.
\begin{equation*}
H(x)v = \lim_{r\rightarrow 0}{\frac{g(x+rv)-g(x-rv)}{2r}}
\end{equation*}

__cntk__에서는 gradient를 구할 때 operator 대신 numerical output이 나오게 되어, Hessian을 구할 때 __numerical approximation__을 사용할 수 밖에 없음.

반면 __tensorflow__에서는 gradient가 operator가 되어, HVP을 tf.gradients를 두번 사용한 operator로 만들 수 있고, 이후 계산은 TF에서 지원해주는 __automatic differentiation__을 사용하기 때문에 정확한 HVP 값을 얻을 수 있다.

## cf) automatic differentiation

refer: 
- https://www.youtube.com/watch?v=pdSCHtPJ4B0
- Automatic Differentiation in Machine Learning: A Survey, JMLR, 2018
    - https://arxiv.org/pdf/1502.05767.pdf

- symbolic differentiation: mathematica 등에서 사용되는 방법으로 sine, cosine 등의 미분식을 이용해서 symbol (x,y 등)에 대한 정확한 미분식을 얻는 방법. 변수가 복잡해질 수록 계산하는 것이 힘들어짐.
- numerical differentiation: $\frac{f(x+h)-f(x)}{h}$를 사용해서 미분값의 approximation을 찾는 방법. h가 작을 수록 미분값에 가까워지지만, round off error에 취약해짐.

automatic differentiation은 SD, ND와 다른 방법론. 언어 내 내장 함수가 주어지면 이를 좀 더 최소 세분화된 node로 나눠 graph를 그리고, 이를 기반으로 미리 정의된 calculus rule을 적용해서 미분을 하는 방법. AD가 계산하는 방법은 forward mode, reverse mode 두 가지가 있으며 서로 장단점이 있음. 이는 각각 예시를 들어가며 설명하도록 함.

e.g.

(1) forward mode

$y = f(x_1,x_2) = ln(x_1) + x_1x_2 - sin(x_2)$

해당 식은 간단해보이지만 사실 더 세분화시켜 computational graph로 표현할 수 있음.

![computational_graph](./images/computational_graph.png)

각 노드가 의미하는 것은 다음 그림에서 확인할 수 있다. 위 node 단위로 나눠지게 되면 단순한 calculus rule을 사용해서 계산을 할 수 있다.

![forward_mode](./images/forward_mode.png)

오른쪽 표는 $x_1$에 대해서 미분을 진행한 것이다. i.e. $\dot{v}_i = \frac{\partial v_i}{\partial x_1}$. 차례로 계산하다보면 최종값($\dot{v}_5$)은 과거의 node로 차근차근 표현해갈 수 있음을 확인할 수 있다. 중간중간 symbolic 미분은 $(a+b)' = a'+b'$나 $(ab)' = a'b + ab'$처럼 단순한 calculus이기 때문에 미리 정의를 해두고 필요시 불러오는 방법으로 진행된다. 때문에 $\dot{v}_{-1}, \dot{v}_{0}$부터 차례대로 계산한다면 최종 미분값까지 도달할 수 있다. y가 scalar가 아니라 vector의 값이더라도 forward pass를 한 번 진행하면 $x_1$에 관한 모든 미분값을 얻을 수 있다는 장점이 있다. 하지만 $x_2$에 대해 미분을 진행하려면 새로이 forward pass를 진행해야한다.
__즉 input dimension이 작고 output dimension이 큰 경우 유리한 방법이다.__

(2) reverse mode

![reverse_mode](./images/reverse_mode.png)

오른쪽 표는 최종 output $y$를 각 node $v_i$에 대해서 미분을 진행한 것이다. i.e. $\bar{v}_i=\frac{\partial y}{\partial v_i}$. 차례로 계산하다보면 최종값 $\bar{x}_1, \bar{x}_2$는 과거의 node로 차근차근 표현해갈 수 있음을 확인할 수 있다. 한 번의 진행과정으로 y를 $x_1, x_2$ 미분값을 동시에 얻을 수 있다. 하지만 y가 scalar가 아니라 vector인 경우 $y_1$, ..., $y_k$에 대해서 각각 진행해야한다. 
__즉 output dimension이 작고 input dimension이 큰 경우 유리한 방법이다.__

$f: R^n \rightarrow R^m$의 경우 Jacobian matrix $J \in R^{m \times n}$을 구할 때

- Forward AD: $O(n \times time(f))$
- Reverse AD: $O(m \times time(f))$

시간이 들게 된다.

In [2]:
def grad_inner_product(grad1, grad2):
    # inner product for dictionary-format gradients (output scalar value)
    
    val = 0
    
    for ks in grad1.keys():
        val += np.sum(np.multiply(grad1[ks],grad2[ks]))
        
    return val

def weight_update(w, v, r):
    # w: weights of neural network (tuple)
    # v: value for delta w (dictionary, e.g., gradient value)
    # r: hyperparameter for a gradient (scalar)

    for p in w:
        p.value += r * v[p]

def HVP(y, x, v):
    # Calculate Hessian vector product 
    # y: scalar function to be differentiated (function, e.g. cross entropy loss)
    # x: feed_dict value for the network (dictionary, e.g. {model.X: image_batch, model.y: label_batch})
    # v: vector to be producted (by Hessian) (numeric dictionary, e.g., g(z_test))
    ## w: variables to differentiate (numeric, e.g. neural network weight)
    
    # hyperparameter r
    r = 1e-2
    
    assert type(x)==dict, "Input of HVP is wrong. this should be dictionary"
     
    w = v.keys()
    
    # gradient for plus
    weight_update(w, v, +r)
    g_plus = y.grad(x, wrt=w)
  
    # gradient for minus
    weight_update(w, v, -2*r)
    g_minus = y.grad(x, wrt=w)
    
    # weight reconstruction
    weight_update(w, v, +r)
    
    hvp = {ks: (g_plus[ks] - g_minus[ks])/(2*r) for ks in g_plus.keys()}
       
    return hvp


In [3]:
# toy example for HVP

x = C.input_variable(shape=(1,))
h = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(x)
y = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(h)

params = y.parameters
x_feed = {x:[[1.]]}
v_feed = {p: np.ones_like(p.value) for p in params}

HVP(y, x_feed, v_feed)

# output should be 1, 1
# tensorflow version with double tf.gradients will outputs exactly (1., 1.) output.

{Parameter('W', [], [1 x 1]): array([[ 0.99999905]], dtype=float32),
 Parameter('W', [], [1 x 1]): array([[ 0.99999905]], dtype=float32)}

In [4]:
# hvp implementation for tensorflow
try:
    import tensorflow as tf
    import numpy as np
    import time

    def HVP(y, x, v):
        # y: scalar loss function
        # w: weight vector in order to generate Hessian
        # x: feed_dict value for input of the network
        # v: vector to be producted (gradient dimension, i.e. list)
        w = tf.trainable_variables(scope='network')

        grads = tf.gradients(y, w)

        grads_inner = grad_inner_product(grads, v)

        hvp = tf.gradients(grads_inner, w)

        hvp_val = list(map(lambda k: k.eval(x),hvp))

        return hvp_val

    def grad_inner_product(grad1, grad2):
        # inner product for list-format gradients (output scalar value)

        val = 0

        for i in range(len(grad1)):
            val += np.sum(np.multiply(grad1[i], grad2[i]))

        return val

    # toy example for HVP

    with tf.variable_scope('network'):
        x = tf.placeholder(tf.float32, (None, 1))
        h = tf.layers.dense(x, 1, activation=None, use_bias=False)
        y = tf.layers.dense(h, 1, activation=None, use_bias=False)

    vars = tf.trainable_variables(scope='network')

    tf.InteractiveSession()
    tf.global_variables_initializer().run()

    x_feed = {x: [[1.]]}
    v_feed = [np.ones_like(p) for p in vars] # gradient_like one vector
    print(v_feed)

    hvp = HVP(y, x_feed, v_feed)
    print('original hvp:', hvp)
    print('concated hvp:', np.concatenate(hvp))

    np.concatenate(hvp).shape

except:
    print('tensorflow is not available')

tensorflow is not available


# Inverse HVP with Conjugate Gradient

Implement conjugate gradient

Conjugate gradient method 방법은 $x^* = H^{-1}v$를 구하고 싶은데, $H$ 차원이 너무 커서 직접적으로 matrix를 계산하고, 역행렬을 얻는 것이 힘들 때 

\begin{equation*}
x^* = \min_{x}{\frac{1}{2}x^T H x - v^T x}
\end{equation*}

를 $Hv$를 반복해서 사용함으로 해를 얻는 방법.

때문에 quadratic programming 형태여야만 이론적 근거를 충족시킴.
(refer: Krylov space)

scipy package에서 제공하는 fmin_ncg는 임의의 convex 함수를 풀 때 conjugate gradient의 iteration 방법을 써서 풀겠다는 것. 주어진 objective function이 convex function 이라면 2nd order Taylor approximation을 할 수 있을 것이고 approximation을 하고 나면 정확하지는 않겠지만 어느정도 비슷하고 convergent한 해를 얻을 수 있다는 것. 따라서 fmin_ncg는 conjugate gradient의 iteration 방법으로 optimization 문제를 풀겠다는 것. 

이렇게 풀 경우에는 Hessian을 직접적으로 구하지 않고 (HVP만 사용해서) Newton's method 방식으로 optimize가 될 것.

cf) 좀 더 general하게 설계되어있다 정도만 알면 됨. 어차피 우리는 objective function에 quadratic function을 넣을거고 그러면 전혀 문제 없음.

Scipy package의 fmin conjugate gradient는 HVP 함수를 explicit하게 제공하면 더 정확하게 풀 수 있음. 따라서 위의 HVP를 사용할 예정.

Theoretical Backgrounds
-----------------------

### Preliminaries

#### Krylov Subspace 

Notations

- $f(x) \overset{def}{=} \frac{1}{2}x^TAx - b^Tx$
- $K_k \overset{def}{=} span\{b,Ab, \ldots, A^{k-1}b\}$; called Krylov subspace
- $x^k \overset{def}{=} \arg{\min_{x\in K_k}{f(x)}}$; called Krylov sequence
- $r^k \overset{def}{=} b - Ax^k = -\nabla f(x^k)$; called residual

Properties

1. $f(x^{k+1}) \le f(x^k)$
2. $x^n = x^*$
3. $x^{k+1} = x^k + \alpha_k r^k + \beta_k (x^k - x^{k-1})$ for some $\alpha_k, \beta_k$

__inverse Hessian vector product를 구할 때 Hessian vector product를 사용한 점화식을 반복하면 해를 얻을 수 있다.__

#### Caley-Hamilton Theorem

$\chi(A) = A^n + \alpha_1 A^{n-1} + \cdots + \alpha_n I = 0$

where $\alpha_i$ comes from characteristic polynomial of A.

This theorem indicates that

$A^{-1} = -\frac{1}{\alpha_n}A^{n-1} - \frac{\alpha_1}{\alpha_n}A^{n-2} - \cdots - \frac{\alpha_{n-1}}{\alpha_n}I$

점화식을 n번 반복하면 정확한 해를 얻을 수 있다. (Property (2) 증명)

#### Settings

Given, 

\begin{align*}
& \hat{\theta} \in \mathbb{R}^p \\
& v = \nabla L(z_{test}, \hat{\theta}) \in \mathbb{R}^p \\
& H = \frac{1}{n}\sum_{i=0}^{n}{\nabla^2 L(z_i,\hat{\theta})} \in \mathbb{R}^{p^2}\\
\end{align*}

Our goal is to find $x^* \rightarrow H^{-1}v$

#### Algorithm

> $x:= 0, r:= v, \rho_0 := \lvert\lvert r \rvert\rvert ^2$

> for $ k=1,\ldots , N_{max} $

>> if $\sqrt{\rho_{k-1}}\le \epsilon \lvert\lvert v \rvert\rvert$ : break

>> if $k=1$ then $p := r$; else $p:=r+\left( \frac{\rho_{k-1}}{\rho_{k-2}} \right) p$

>> $w := Hp$

>> $\alpha := \frac{\rho_{k}}{p^Tw}$

>> $x:=x+\alpha p$

>> $r := r-\alpha w$

>> $\rho_{k}:=\lvert\lvert r \rvert\rvert ^2$

Remark) 
- $w := Hp$ 진행할 때 HVP를 사용하면 $O(np)$만큼 computational complexity 듬.
- 실제 code에서는 모든 training dataset에 대해서 HVP를 하려면 한번에 모든 training set에 대해서 feed_dict해줘야하기 때문에 메모리 문제로 HVP_minibatch를 구현함. (밑 코드 참고)


In [6]:
from scipy.optimize import fmin_ncg

def HVP_minibatch_val(model, y, v, data_set):
    # Calculate Hessian vector product w.r.t whole dataset
    # model: neural network model (e.g. model)
    # y: scalar function output of the neural network (e.g. model.loss)
    # v: vector to be producted by inverse hessian (i.e.H^-1 v) (numeric dictionary, e.g. v_test)
    # data_set: training set to be summed in Hessian
    
    # hyperparameters
    damping = 0.0 # convexity term; paper ref:0.01
    batch_size = 1
    
    hvp_batch = {ks: [] for ks in v.keys()}
    
    dataloader = DataLoader(data_set, batch_size, shuffle=True, num_workers=6)
    
    for img, lb in dataloader:
        img = img.numpy(); lb = lb.numpy()
        x_feed = {model.X: img, model.y:lb}
        hvp = HVP(y,x_feed,v)
        # add hvp value
        [hvp_batch[ks].append(hvp[ks]/img.shape[0]) for ks in hvp.keys()]
        
    hvp_mean = {ks: np.mean(hvp_batch[ks], axis=0) + damping*v[ks] for ks in hvp_batch.keys()}
    
    return hvp_mean

# x: solution vector for conjugate gradient, whose shape is same as flattened gradient. NOT feed dict value

def dic2vec(dic):
    # convert a dictionary with matrix values to a 1D vector
    # e.g. gradient of network -> 1D vector
    vec = np.concatenate([val.reshape(-1) for val in dic.values()])
    
    return vec

def vec2dic(vec, fmt):
    # convert a 1D vector to a dictionary of format fmt
    # fmt = {key: val.shape for (key,val) in dict}
    
    fmt_idx = [np.prod(val) for val in fmt.values()]
    #lambda ls, idx: [ls[sum(idx[:i]):sum(idx[:i+1])] for i in range(len(idx))]
    vec_split = [vec[sum(fmt_idx[:i]):sum(fmt_idx[:i+1])] for i in range(len(fmt_idx))]
    dic = {key: vec_split[i].reshape(shape) for (i,(key,shape)) in enumerate(fmt.items())}

    return dic


def get_fmin_loss_fn(model, y, v, data_set):
    
    def get_fmin_loss(x):
        x_dic = vec2dic(x, {key: val.shape for (key, val) in v.items()})
        hvp_val = HVP_minibatch_val(model, y, x_dic, data_set)
        
        return 0.5 * grad_inner_product(hvp_val, x_dic) - grad_inner_product(v, x_dic)
    
    return get_fmin_loss

def get_fmin_grad_fn(model, y, v, data_set):
    
    def get_fmin_grad(x):
        # x: 1D vector
        x_dic = vec2dic(x, {key: val.shape for (key, val) in v.items()})
        hvp_val = HVP_minibatch_val(model, y, x_dic, data_set)
        hvp_flat = dic2vec(hvp_val)
        v_flat = dic2vec(v)
        
        return hvp_flat - v_flat
    
    return get_fmin_grad

def get_fmin_hvp_fn(model, y, v, data_set):

    def get_fmin_hvp(x, p):
        p_dic = vec2dic(p, {key: val.shape for (key, val) in v.items()})
        hvp_val = HVP_minibatch_val(model, y, p_dic, data_set)
        hvp_flat = dic2vec(hvp_val)

        return hvp_flat
    
    return get_fmin_hvp

def get_inverse_hvp_cg(model, y, v, data_set):
    # return x, which is the solution of QP, whose value is H^-1 v
    
    fmin_loss_fn = get_fmin_loss_fn(model, y, v, data_set)
    fmin_grad_fn = get_fmin_grad_fn(model, y, v, data_set)
    fmin_hvp_fn = get_fmin_hvp_fn(model, y, v, data_set)
    
    fmin_results = fmin_ncg(\
            f = fmin_loss_fn,\
            x0 = dic2vec(v),\
            fprime = fmin_grad_fn,\
            fhess_p = fmin_hvp_fn,\
            avextol = 1e-8,\
            maxiter = 1e2)
    
    return fmin_results
    #return vec2dic(fmin_results, {key: val.shape for (key, val) in v.items()}))


scipy의 conjugate gradient을 이용해서 구현하였다. 

fmin_ncg의 input으로는

- f: objective function to be minimized ~ $f(x)=\frac{1}{2}x^T H x - v^T x$ where $H\triangleq \frac{1}{n}\sum_{i=0}^{n}{\nabla^2 L(z_i,x)}$
- x0: initial guess
- fprime: gradient of f ~ $f(x)=H x - v$
- fhess_p: function to compute the Hessian matrix of f ~ $f(x,p)=H p$
- callback: an optional user-supplied function which is called after each iteration
- avextol: hyperparameter epsilon 
- maxiter: maximum number of iterations to perform

이 있다.


# Toy Example

- __Set__ Training Set: 
$\mathbb{T} = \{(2,1)\}$

- Network Architecture: 
$\hat{y} = w_1 w_2 x$

- Loss Function: 
$L(w) = \sum_{(x^{(i)},y^{(i)})\in\mathbb{T}}{(y^{(i)}_1 - (w_1 w_2 x^{(i)}))^2} = 4w_1^2w_2^2 -4w_1w_2 +1$

- Hessian: 
$
H(w) = \left[ {\begin{array}{cc}
   8w_2^2  & 16w_1w_2-4 \\
   16w_1w_2-4 & 8w_1^2 \\
  \end{array} } \right] \\
$

cf) check the graph here

https://www.google.co.kr/search?ei=evtoW5m5JpTe8wWggLqAAQ&q=%281-2*x*y%29%5E2&oq=%281-2*x*y%29%5E2&gs_l=psy-ab.3..0i7i30k1j0i30k1j0i7i5i30k1j0i8i30k1l4j0i5i30k1l2j0i8i30k1.5016.5780.0.6095.2.2.0.0.0.0.129.249.0j2.2.0....0...1c.1.64.psy-ab..0.2.248...0i8i7i30k1.0.tJbFx39Pz_s

For some $(w_1,w_2)$, loss function is not convex.

- __Set__ Network Parameter: 
$(w_1,w_2) = (1,\frac{1}{3})$

- Value of Hessian:
$
H = H((w_1,w_2)=(1,\frac{1}{3})) = \left[ {\begin{array}{cc}
   \frac{8}{9}  & \frac{4}{3} \\
   \frac{4}{3} & 8 \\
  \end{array} } \right] \\
$

- Characteristic polynomial of H:
$\det{(H-\lambda I)} = \lambda^2 - \frac{80}{9}\lambda + \frac{48}{9}$

indicating that $L$ is locally convex under $(w_1,w_2) = (1,\frac{1}{3})$.

- Inverse Hessian:
$H^{-1} = \left[ {\begin{array}{cc}
   \frac{3}{2}  & -\frac{1}{4} \\
   -\frac{1}{4} & \frac{1}{6} \\
  \end{array} } \right] \\$
  
- __Set__ vector:
$ v = [1., 1.]^T $

- Inverse Hessian Vector Product:
$ H^{-1}v = [\frac{5}{4}, -\frac{1}{12}]^T \approx [1.25, -0.083]^T $

In [7]:
# toy example for IHVP: 1D example

class SimpleNet(object):
    def __init__(self):
        self.X = C.input_variable(shape=(1,))
        self.h = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.X)
        self.pred = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.h)
        self.y = C.input_variable(shape=(1,))
        self.loss = C.squared_error(self.pred, self.y)
        
class SimpleDataset(object):
    def __init__(self, images, labels):
        self._images, self._labels = images, labels
    
    def __getitem__(self, index):
        X = self._images[index]
        y = self._labels[index]
        
        return X, y
    
    def __len__(self):
        return len(self._images)


net = SimpleNet()

params = net.pred.parameters

x_feed = {net.X:np.array([[2.]],dtype=np.float32), net.y:np.array([[1.]],dtype=np.float32)}
v_feed = {p: np.ones_like(p.value) for p in params}

print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))
params[0].value = np.asarray([[1.]])
params[1].value = np.asarray([[1./3.]])
print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))

print('hvp', HVP(net.loss, x_feed, v_feed))

images = np.asarray([[2.],[2.]], dtype=np.float32)
labels = np.asarray([[1.],[1.]], dtype=np.float32)
#images = np.asarray([[2.]], dtype=np.float32)
#labels = np.asarray([[1.]], dtype=np.float32)

train_set = SimpleDataset(images,labels)

print('hvp_batch', HVP_minibatch_val(net, net.loss, v_feed, train_set))

print('inverse hvp', get_inverse_hvp_cg(net, net.loss, v_feed, train_set))

w1 = 
 [[ 0.11753198]] 
w2 = 
 [[ 0.80720806]] 
loss = 
 [ 0.65651226]
w1 = 
 [[ 1.]] 
w2 = 
 [[ 0.33333334]] 
loss = 
 [ 0.1111111]
hvp {Parameter('W', [], [1 x 1]): array([[ 2.22302079]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 9.33413506]], dtype=float32)}
hvp_batch {Parameter('W', [], [1 x 1]): array([[ 2.22302151]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 9.33413506]], dtype=float32)}
         Current function value: -0.583326
         Iterations: 4
         Function evaluations: 22
         Gradient evaluations: 14
         Hessian evaluations: 7
inverse hvp [ 1.24996972 -0.08331382]


# Inverse HVP with Stochastic Estimation

Implementation of IHVP with SE.

Theoretical Backgrounds
-----------------------

Taylor series expansion을 통해서 

\begin{equation*}
H_j^{-1}\triangleq \sum_{i=0}^{j}{(I-H)^i}
\end{equation*}

Hessian matrix의 inverse를 approximate할 수 있다. 이 식을 새로 정리하면 다음과 같은 점화식을 얻을 수 있다.

\begin{equation*}
H_j^{-1} = I + (I-H)H_{j-1}^{-1}
\end{equation*}

우리는 이 점화식을 반복하여 진행하며 Hessian의 inverse를 추정할 것이다. 이 점화식을 반복하기 위해서는 H를 계산해야한다. 이 때 H는 $\frac{1}{n}\sum_{i=0}^n{\nabla^2L(z_i,\hat{\theta}})$ 값으로 이 값 정확하게 구하려면 지나치게 많은 계산량을 필요로 한다. 때문에 이를 unbiased estimator인 $\nabla^2L(z_i,\hat{\theta})$로 대처하여 iteration을 진행할 것이라는게 논문의 내용이다. 이 때 unbiased estimator of Hessian을 $\tilde{H}$라고 하자. 실제 구현 코드에서는 unbiased estimator $\tilde{H}$를 좀 더 잘 (낮은 variance로 추정하기) 구하기 위해서 몇 가지 방법을 제시하고 있다.

#### Minibatching
하나의 sample만을 가지고 unbiased estimator를 사용하는 대신, 여러 sampling에 대해서 평균취하는 방법이다. 평균을 취했기 때문에 mean값에는 변화가 없고, variance는 줄어들 것으로 기대할 수 있다. 이 때 sample 수는 $m$개이고 당연히 $m<<n$이다. e.g., Gaussian random variable이라고 가정할 경우 mean은 변함없고, variance는 $\frac{1}{m^2}$로 줄어들 것이다. 

\begin{eqnarray*}
\tilde{H} 
&=& \nabla^2L(z_i,\hat{\theta})\\
&\rightarrow& \frac{1}{m}\sum_{i=0}^m{\nabla^2L(z\_i,\hat{\theta}})
\end{eqnarray*}

#### Scaling
H를 추정하는 대신 $H'\triangleq\frac{H}{Scale}$을 추정하는 것이다. 상수배로 나뉘었기 때문에 variance가 줄어들 것이라고 예측할 수 있다. e.g., Gaussian random variable이라고 가정할 경우 variance가 $\frac{1}{Scale^2}$ 만큼 줄어들 것이다. 이는 minibatching과 다르게 하나의 step에서 실질적인 계산량은 변함없이 정확도를 향상시킬 수 있다. 하지만 수렴을 위해서 더 많은 recursion step (j)을 요구한다. 때문에 scaling factor는 learning rate와도 연관지어 생각할 수 있다.

\begin{eqnarray*}
{H'}_j^{-1} 
&=& I + (I-H'){H'}_{j-1}^{-1}\\
&=& I + (I-\frac{H}{Scale}){H'}_{j-1}^{-1}\\
\end{eqnarray*}

그리고 이 값은 ${H'}_j^{-1}=\left(\frac{H_j}{Scale}\right)^{-1}$를 추정한 것이기 때문에 $H_j^{-1}$을 얻기위해선 iteration을 통해 얻은 최종 값을 Scale값으로 나눠야한다.

cf) Talor expansion이기 때문에 initialization 값은 변함이 없다, i.e., ${H'}_0^{-1}=I$.

#### Damping
마지막으로 code에는 damping term이 있다. 이 term은 앞서 Influence function을 얻을 때 strictly convex와 twice differentiable이라는 두 조건 중 strictly convex 조건을 더 강력하게 맞추기위해서 생겨난 regularization term이다. 특히 deep neural network 같은 경우에는 비용함수가 strictly convex라는 보장이 전혀 없기 때문에 꼭 필요한 term이다.

정확하게 말하자면 positive eigenvalue를 얻기위해 임의로 identity matrix를 더해주는 것으로 수식으로 표현하자면 다음과 같다.

\begin{equation*}
\bar{H}=\tilde{H}+\lambda I
\end{equation*}

이 $\bar{H}$를 점화식에 넣게 되면 다음과 같은 식을 얻을 수 있다.

\begin{eqnarray*}
H_j^{-1} 
&=& I + (I-\bar{H})H_{j-1}^{-1}\\
&=& I + \left((1-\lambda)I-\tilde{H}\right)H_{j-1}^{-1}\\
\end{eqnarray*}


Implementations
---------------
이 방법을 통해서 찾고자하는 최종 값은 $\theta$와 차원이 같은 임의의 vector $v$와 Inverse of Hessian matrix의 곱, $H^{-1}v$ 값이다.
따라서 위 점화식의 양 변에 $v$를 곱하고 추정할 vector를 $s_j$로 새로 정의하면, $\text{i.e., } s_j \triangleq H_j^{-1}v$, 다음과 같은 반복식을 얻을 수 있다.

\begin{equation*}
s_j = v + s_{j-1} - \bar{H} s_{j-1}
\end{equation*}

Strictly convex 조건을 만족하기위해서 damping term을 풀면 다음과 같은 식을 얻을 수 있다.

\begin{equation*}
s_j = v + (1-\gamma)s_{j-1} - \tilde{H} s_{j-1}
\end{equation*}

위에서 설명한 minibatching과 scaling을 포함하여 점화식을 얻으면 다음과 같다.

\begin{equation*}
{s'}_j = v + (1-\gamma){s'}_{j-1} - \frac{\frac{1}{m}\sum_{i=0}^m{\nabla^2L(z\_i,\hat{\theta}})}{Scale} {s'}_{j-1} \text{ where } {s'}_j = Scale \cdot s_j 
\end{equation*}

실제 구현된 코드의 input과 theoretical background의 hyperparameter들과 관련하여 설명하자면

1. $\gamma \leftarrow$ damping : logistic regression 같이 strictly convex한 loss를 사용할 경우 0.0으로 두면 된다. 논문에서는 $\lambda=0.01$로 사용해서 CNN network를 재 조정했다. 

2. $Scale \leftarrow$ scale : 앞서 설명했듯 H에 대해서 iteration을 반복할 때 H를 scale로 나눠서 진행하는 것. vector s의 변화량이 줄어들기 때문에 variance가 줄어들 수 있겠지만 더 많은 recursion step이 필요할 것이다. recursion step을 거쳐서 얻은 후 다시 scale로 나누는 것을 잊지 말 것.

3. $m \leftarrow$ batch_size : 여러 sample을 사용하여 평균취하는 것으로 unbiased estimator를 얻은 것. 클 수록 좋겠지만 rtp * batch_size만큼 계산량이 느는 것이라 주의해야함. 실제 구현 단계에서는 total loss 자체가 minibatch에 대해서 reduce sum한 것이기 때문에 feed_dict만 minibatch wise로 하면 된다.

$O(rtp)$

$r$: num_samples

$t$: recursion_depth

$p$: dimension of $\theta$


In [9]:
# stochastic estimation
def get_inverse_hvp_se(model, y, v, data_set, **kwargs):
    # Calculate inverse hessian vector product over the training set
    # model: neural network model (e.g. model)
    # y: scalar function output of the neural network (e.g. model.loss)
    # v: vector to be producted by inverse hessian (i.e.H^-1 v) (e.g. v_test)
    # data_set: training set to be summed in Hessian
    # kwargs: hyperparameters for stochastic estimation
    recursion_depth = kwargs.pop('recursion_depth', 50) # epoch
    scale = kwargs.pop('scale', 1e1) # similar to learning rate
    damping = kwargs.pop('damping', 0.0) # paper reference: 0.01
    batch_size = kwargs.pop('batch_size', 1)
    num_samples = kwargs.pop('num_samples', 1) # the number of samples(:stochatic estimation of IF) to be averaged
    verbose = kwargs.pop('verbose', False)
    
    dataloader = DataLoader(data_set, batch_size, shuffle=True, num_workers=6)
    
    inv_hvps = []
    
    params = v.keys()
    
    for i in range(num_samples):
        # obtain num_samples inverse hvps
        cur_estimate = v
        
        for depth in range(recursion_depth):
            # epoch-scale recursion depth
            t1 = time.time()
            for img, lb in dataloader:
                img = img.numpy(); lb = lb.numpy()
                x_feed = {model.X: img, model.y:lb}
                hvp = HVP(y,x_feed,cur_estimate)
                # cur_estimate = v + (1-damping)*cur_estimate + 1/scale*(hvp/batch_size)
                cur_estimate = {ks: v[ks] + (1-damping)*cur_estimate[ks] - (1/scale)*hvp[ks]/batch_size for ks in cur_estimate.keys()}
            if verbose:
                print('#w: \n', list(map(lambda x: x.value, params)), '\n#hvp: \n', hvp, '\n#ihvp: \n', cur_estimate)
                print("Recursion depth: {}, norm: {}, time: {} \n".format(depth, np.sqrt(grad_inner_product(cur_estimate,cur_estimate)),time.time()-t1))
        
        inv_hvp = {ks: (1/scale)*cur_estimate[ks] for ks in cur_estimate.keys()}
        inv_hvps.append(inv_hvp)
    
    inv_hvp_val = {ks: np.mean([inv_hvps[i][ks] for i in range(num_samples)], axis=0) for ks in inv_hvps[0].keys()}
    
    return inv_hvp_val

In [10]:
# toy example for IHVP: 1D example

class SimpleNet(object):
    def __init__(self):
        self.X = C.input_variable(shape=(1,))
        self.h = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.X)
        self.pred = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.h)
        self.y = C.input_variable(shape=(1,))
        #self.loss = C.reduce_l2(self.pred-self.y)
        self.loss = C.squared_error(self.pred, self.y)
        
class SimpleDataset(object):
    def __init__(self, images, labels):
        self._images, self._labels = images, labels
    
    def __getitem__(self, index):
        X = self._images[index]
        y = self._labels[index]
        
        return X, y
    
    def __len__(self):
        return len(self._images)


net = SimpleNet()

params = net.pred.parameters

x_feed = {net.X:np.array([[2.]],dtype=np.float32), net.y:np.array([[1.]],dtype=np.float32)}
v_feed = {p: np.ones_like(p.value) for p in params}

print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))
params[0].value = np.asarray([[1.]])
params[1].value = np.asarray([[1./3.]])
print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))

print('hvp', HVP(net.loss, x_feed, v_feed))

images = np.asarray([[2.],[2.]], dtype=np.float32)
labels = np.asarray([[1.],[1.]], dtype=np.float32)
#images = np.asarray([[2.]], dtype=np.float32)
#labels = np.asarray([[1.]], dtype=np.float32)

train_set = SimpleDataset(images,labels)

print('inverse hvp', get_inverse_hvp_se(net, net.loss, v_feed, train_set))

w1 = 
 [[ 0.3461298]] 
w2 = 
 [[ 0.5710966]] 
loss = 
 [ 0.36560512]
w1 = 
 [[ 1.]] 
w2 = 
 [[ 0.33333334]] 
loss = 
 [ 0.1111111]
hvp {Parameter('W', [], [1 x 1]): array([[ 2.22302079]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 9.33413506]], dtype=float32)}
inverse hvp {Parameter('W', [], [1 x 1]): array([[ 1.24517536]], dtype=float32), Parameter('W', [], [1 x 1]): array([[-0.08125092]], dtype=float32)}


In [11]:
# graphical analysis of stochastic estimation 

def plot_inverse_hvp_se(model, y, v, data_set, **kwargs):
    # Calculate inverse hessian vector product over the training set
    # AND plot the process of convergence w.r.t. 2D space
    # model: neural network model (e.g. model)
    # y: scalar function output of the neural network (e.g. model.loss)
    # v: vector to be producted by inverse hessian (i.e.H^-1 v) (e.g. v_test)
    # data_set: training set to be summed in Hessian
    # kwargs: hyperparameters for stochastic estimation
    recursion_depth = kwargs.pop('recursion_depth', 100) # epoch
    scale = kwargs.pop('scale', 1e1) # similar to learning rate
    damping = kwargs.pop('damping', 0.0) # paper reference: 0.01
    batch_size = kwargs.pop('batch_size', 1)
    num_samples = kwargs.pop('num_samples', 1) # the number of samples(:stochatic estimation of IF) to be averaged
    verbose = kwargs.pop('verbose', False)
    graphics = kwargs.pop('graphics', True)
    
    dataloader = DataLoader(data_set, batch_size, shuffle=True, num_workers=6)
    
    inv_hvps = []
    
    pts = [] # graphical points
    # pts.append(dic2vec(v))
    
    params = y.parameters
    
    for i in range(num_samples):
        # obtain num_samples inverse hvps
        cur_estimate = v
        
        for depth in range(recursion_depth):
            # epoch-scale recursion depth
            t1 = time.time()
            for img, lb in dataloader:
                img = img.numpy(); lb = lb.numpy()
                x_feed = {model.X: img, model.y:lb}
                hvp = HVP(y,x_feed,cur_estimate)
                # cur_estimate = v + (1-damping)*cur_estimate + 1/scale*(hvp/batch_size)
                cur_estimate = {ks: v[ks] + (1-damping)*cur_estimate[ks] - (1/scale)*hvp[ks]/batch_size for ks in cur_estimate.keys()}
            if verbose:
                print('#w: \n', list(map(lambda x: x.value, params)), '\n#hvp: \n', hvp, '\n#ihvp: \n', cur_estimate)
                print("Recursion depth: {}, norm: {}, time: {} \n".format(depth, np.sqrt(grad_inner_product(cur_estimate,cur_estimate)),time.time()-t1))
            if graphics:
                curv = dic2vec(cur_estimate)/scale
                if np.isnan(curv).any() or (np.abs(curv)>1e1).any():
                    pts.append(1e1*np.ones_like(curv))
                else:
                    pts.append(curv)

        inv_hvp = {ks: (1/scale)*cur_estimate[ks] for ks in cur_estimate.keys()}
        inv_hvps.append(inv_hvp)
    
    inv_hvp_val = {ks: np.mean([inv_hvps[i][ks] for i in range(num_samples)], axis=0) for ks in inv_hvps[0].keys()}
    
    return inv_hvp_val, pts

# toy example for IHVP: 1D example

class SimpleNet(object):
    def __init__(self):
        self.X = C.input_variable(shape=(1,))
        self.h = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.X)
        self.pred = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.h)
        self.y = C.input_variable(shape=(1,))
        self.loss = C.squared_error(self.pred, self.y)
        
class SimpleDataset(object):
    def __init__(self, images, labels):
        self._images, self._labels = images, labels
    
    def __getitem__(self, index):
        X = self._images[index]
        y = self._labels[index]
        
        return X, y
    
    def __len__(self):
        return len(self._images)


net = SimpleNet()

params = net.pred.parameters

x_feed = {net.X:np.array([[2.]],dtype=np.float32), net.y:np.array([[1.]],dtype=np.float32)}
v_feed = {p: np.ones_like(p.value) for p in params}

print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))
params[0].value = np.asarray([[1.]])
params[1].value = np.asarray([[1./3.]])
print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))

print('hvp', HVP(net.loss, x_feed, v_feed))

#images = np.asarray([[2.],[2.]], dtype=np.float32)
#labels = np.asarray([[1.],[1.]], dtype=np.float32)
images = np.asarray([[2.]], dtype=np.float32)
labels = np.asarray([[1.]], dtype=np.float32)

train_set = SimpleDataset(images,labels)

tmp, pts_scale20 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'scale':20})
print('scale20:', tmp)
tmp, pts_scale10 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'scale':10})
print('scale10:', tmp)
tmp, pts_scale5 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'scale':5})
print('scale5:', tmp)
tmp, pts_scale1 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'scale':1})
print('scale1:', tmp)

#print(pts_scale1)

w1 = 
 [[ 0.50877059]] 
w2 = 
 [[ 0.55213112]] 
loss = 
 [ 0.19200508]
w1 = 
 [[ 1.]] 
w2 = 
 [[ 0.33333334]] 
loss = 
 [ 0.1111111]
hvp {Parameter('W', [], [1 x 1]): array([[ 2.22302079]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 9.33413506]], dtype=float32)}
scale20: {Parameter('W', [], [1 x 1]): array([[ 1.19857621]], dtype=float32), Parameter('W', [], [1 x 1]): array([[-0.07018896]], dtype=float32)}
scale10: {Parameter('W', [], [1 x 1]): array([[ 1.24517429]], dtype=float32), Parameter('W', [], [1 x 1]): array([[-0.08125068]], dtype=float32)}
scale5: {Parameter('W', [], [1 x 1]): array([[ 1.24909544]], dtype=float32), Parameter('W', [], [1 x 1]): array([[-0.08285888]], dtype=float32)}


/home/hwehee/anaconda3/envs/cntk/lib/python3.5/site-packages/ipykernel/__main__.py:17: RuntimeWarning: invalid value encountered in add


scale1: {Parameter('W', [], [1 x 1]): array([[ nan]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ nan]], dtype=float32)}


In [ ]:
import matplotlib.pyplot as plt

x0 = -0.01; y0 = 0.01

# total
plt.figure(figsize=(20,10))

for i in range(len(pts_scale10)):
    blk, = plt.plot(*pts_scale20[i], 'k.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale20[i]+np.array([x0,y0])), str(i), color='k')

    blu, = plt.plot(*pts_scale10[i], 'b.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale10[i]+np.array([x0,y0])), str(i), color='b')

    red, = plt.plot(*pts_scale5[i], 'r.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale5[i]+np.array([x0,y0])), str(i), color='r')

    grn, = plt.plot(*pts_scale1[i], 'g.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale1[i]+np.array([x0,y0])), str(i), color='g')

    
plt.axis([0, 1.5, -0.15, 0.2]) # [xmin, xmax, ymin, ymax]
plt.legend((blk,blu,red,grn), ('scale20','scale10','scale5','scale1'))

plt.savefig('./images/result.png')
plt.show()

# scale20
plt.figure(figsize=(20,10))

for i in range(len(pts_scale10)):
    plt.plot(*pts_scale20[i], 'k.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale20[i]+np.array([x0,y0])), str(i), color='k')

plt.axis([0, 1.5, -0.15, 0.2]) # [xmin, xmax, ymin, ymax]

plt.savefig('./images/result_scale20.png')
plt.show()

# scale10
plt.figure(figsize=(20,10))

for i in range(len(pts_scale10)):
    plt.plot(*pts_scale10[i], 'b.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale10[i]+np.array([x0,y0])), str(i), color='b')

plt.axis([0, 1.5, -0.15, 0.2]) # [xmin, xmax, ymin, ymax]

plt.savefig('./images/result_scale10.png')
plt.show()

# scale5
plt.figure(figsize=(20,10))

for i in range(len(pts_scale10)):
    plt.plot(*pts_scale5[i], 'r.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale5[i]+np.array([x0,y0])), str(i), color='r')

plt.axis([0, 1.5, -0.15, 0.2]) # [xmin, xmax, ymin, ymax]

plt.savefig('./images/result_scale5.png')
plt.show()

# scale1
plt.figure(figsize=(20,10))

for i in range(len(pts_scale10)):
    plt.plot(*pts_scale1[i], 'g.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_scale1[i]+np.array([x0,y0])), str(i), color='g')

    
plt.axis([0, 1.5, -0.15, 0.2]) # [xmin, xmax, ymin, ymax]

plt.savefig('./images/result_scale1.png')
plt.show()

In [ ]:
# do this w.r.t. damping
# graphical analysis of stochastic estimation 

def plot_inverse_hvp_se(model, y, v, data_set, **kwargs):
    # Calculate inverse hessian vector product over the training set
    # AND plot the process of convergence w.r.t. 2D space
    # model: neural network model (e.g. model)
    # y: scalar function output of the neural network (e.g. model.loss)
    # v: vector to be producted by inverse hessian (i.e.H^-1 v) (e.g. v_test)
    # data_set: training set to be summed in Hessian
    # kwargs: hyperparameters for stochastic estimation
    recursion_depth = kwargs.pop('recursion_depth', 100) # epoch
    scale = kwargs.pop('scale', 1e1) # similar to learning rate
    damping = kwargs.pop('damping', 0.0) # paper reference: 0.01
    batch_size = kwargs.pop('batch_size', 1)
    num_samples = kwargs.pop('num_samples', 1) # the number of samples(:stochatic estimation of IF) to be averaged
    verbose = kwargs.pop('verbose', False)
    graphics = kwargs.pop('graphics', True)
    
    dataloader = DataLoader(data_set, batch_size, shuffle=True, num_workers=6)
    
    inv_hvps = []
    
    pts = [] # graphical points
    # pts.append(dic2vec(v))
    
    params = y.parameters
    
    for i in range(num_samples):
        # obtain num_samples inverse hvps
        cur_estimate = v
        
        for depth in range(recursion_depth):
            # epoch-scale recursion depth
            t1 = time.time()
            for img, lb in dataloader:
                img = img.numpy(); lb = lb.numpy()
                x_feed = {model.X: img, model.y:lb}
                hvp = HVP(y,x_feed,cur_estimate)
                # cur_estimate = v + (1-damping)*cur_estimate + 1/scale*(hvp/batch_size)
                cur_estimate = {ks: v[ks] + (1-damping)*cur_estimate[ks] - (1/scale)*hvp[ks]/batch_size for ks in cur_estimate.keys()}
            if verbose:
                print('#w: \n', list(map(lambda x: x.value, params)), '\n#hvp: \n', hvp, '\n#ihvp: \n', cur_estimate)
                print("Recursion depth: {}, norm: {}, time: {} \n".format(depth, np.sqrt(grad_inner_product(cur_estimate,cur_estimate)),time.time()-t1))
            if graphics:
                curv = dic2vec(cur_estimate)/scale
                if np.isnan(curv).any() or (np.abs(curv)>1e1).any():
                    pts.append(1e1*np.ones_like(curv))
                else:
                    pts.append(curv)

        inv_hvp = {ks: (1/scale)*cur_estimate[ks] for ks in cur_estimate.keys()}
        inv_hvps.append(inv_hvp)
    
    inv_hvp_val = {ks: np.mean([inv_hvps[i][ks] for i in range(num_samples)], axis=0) for ks in inv_hvps[0].keys()}
    
    return inv_hvp_val, pts

# toy example for IHVP: 1D example

class SimpleNet(object):
    def __init__(self):
        self.X = C.input_variable(shape=(1,))
        self.h = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.X)
        self.pred = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.h)
        self.y = C.input_variable(shape=(1,))
        self.loss = C.squared_error(self.pred, self.y)
        
class SimpleDataset(object):
    def __init__(self, images, labels):
        self._images, self._labels = images, labels
    
    def __getitem__(self, index):
        X = self._images[index]
        y = self._labels[index]
        
        return X, y
    
    def __len__(self):
        return len(self._images)


net = SimpleNet()

params = net.pred.parameters

x_feed = {net.X:np.array([[2.]],dtype=np.float32), net.y:np.array([[1.]],dtype=np.float32)}
v_feed = {p: np.ones_like(p.value) for p in params}

print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))
params[0].value = np.asarray([[1.]])
params[1].value = np.asarray([[1./3.]])
print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))

print('hvp', HVP(net.loss, x_feed, v_feed))

images = np.asarray([[2.]], dtype=np.float32)
labels = np.asarray([[1.]], dtype=np.float32)

train_set = SimpleDataset(images,labels)
tmp, pts_damp00 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':0.0})
print('damp00:', tmp)
tmp, pts_damp01 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':0.1})
print('damp01:', tmp)
tmp, pts_damp05 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':0.5})
print('damp05:', tmp)
tmp, pts_damp10 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':1.0})
print('damp10:', tmp)

In [ ]:
import matplotlib.pyplot as plt

x0 = -0.01; y0 = 0.01

# total
plt.figure(figsize=(20,10))

for i in range(len(pts_damp00)):
    blk, = plt.plot(*pts_damp00[i], 'k.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_damp00[i]+np.array([x0,y0])), str(i), color='k')

    blu, = plt.plot(*pts_damp01[i], 'b.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_damp01[i]+np.array([x0,y0])), str(i), color='b')

    red, = plt.plot(*pts_damp05[i], 'r.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_damp05[i]+np.array([x0,y0])), str(i), color='r')

    grn, = plt.plot(*pts_damp10[i], 'g.', alpha=min((10+i)/len(pts),1.))
    plt.text(*(pts_damp10[i]+np.array([x0,y0])), str(i), color='g')
    
plt.axis([0, 1.5, -0.15, 0.2]) # [xmin, xmax, ymin, ymax]
plt.legend((blk,blu,red,grn), ('damp0.0', 'damp0.1', 'damp0.5', 'damp1.0'))

plt.savefig('./images/result_damp.png')
plt.show()

## Experimental Results for scale & damping parameters

### Scale

![](./images/result_scale.png)
![](./images/result_scale20.png)
![](./images/result_scale10.png)
![](./images/result_scale5.png)
![](./images/result_scale1.png)

- scale 값이 1.0인 경우를 제외하고는 모두 우리가 원했던 해인 (1.25, -0.083)에 도달했다. 
- scale이 크면 클 수록 해당 값에 도달하기 위해서 더 많은 step 수가 필요했음을 확인할 수 있다.
- scale이 작으면 작을 수록 목표 값에 빨리 도달하나, 더 불안해지는 것을 확인할 수 있었다. 특히 scale이 1일 때에는 3,4 step만에 발산해버린다.
- 결론: scale은 (중간에서 발산하는 경우가 아니라면) 최종 해를 변화시키지 않으며, 크면 클 수록 안정적으로 수렴하나 더 많은 step을 요구하게 된다. (즉 learning rate라고 생각하면 됨.)

### Damping

![](./images/result_damp.png)

- 여러 damp값에 대해서 모두 잘 수렴하였다.
- 주의할 점은 damp term이 $\bar{H} = H'+\gamma I$인데, $H'=\frac{H}{scale}$이라서 scale과도 연관이 있다.
    - 정확하게 설명하자면 damp * scale 만큼의 효과가 있다.
    - scale은 learning rate처럼 동작해야하는데, damping term을 추가하게 되면 그렇지 않게 된다. 따라서 최종적으로는 __(원저자의 코드와 달리) damping_new = damping/scale를 사용__해서 구현하기로 했다.
- 따라서 0.0, 0.1, 0.5, 1.0을 가지고 damping을 추가하면 사실상 0.0, 1.0, 5.0, 10.0의 damping term이 들어간 효과가 나게 된다.
- 이를 가지고 numerical하게 $(H+\gamma I)^{-1} v$ 값을 계산하면, (1.25, -0.083), (0.5036, 0.0365), (0.1560, 0.0609), (0.0858, 0.0492) 값이 나오게 된다.
- 모든 결과 예측한 수치값에 잘 수렴한 것을 확인할 수 있다.
- 하지만 damp term을 추가하게 되면, 이 값이 크면 클 수록 inverse hessian vector product 값도 바뀔 가능성이 커진다는 것을 유의해야 한다.


In [12]:
# stochastic estimation
def get_inverse_hvp_se(model, y, v, data_set, **kwargs):
    # Calculate inverse hessian vector product over the training set
    # model: neural network model (e.g. model)
    # y: scalar function output of the neural network (e.g. model.loss)
    # v: vector to be producted by inverse hessian (i.e.H^-1 v) (e.g. v_test)
    # data_set: training set to be summed in Hessian
    # kwargs: hyperparameters for stochastic estimation
    recursion_depth = kwargs.pop('recursion_depth', 50) # epoch
    scale = kwargs.pop('scale', 1e1) # similar to learning rate
    damping = kwargs.pop('damping', 0.0) # paper reference: 0.01
    batch_size = kwargs.pop('batch_size', 1)
    num_samples = kwargs.pop('num_samples', 1) # the number of samples(:stochatic estimation of IF) to be averaged
    verbose = kwargs.pop('verbose', False)
    
    dataloader = DataLoader(data_set, batch_size, shuffle=True, num_workers=6)
    
    inv_hvps = []
    
    params = y.parameters
    
    for i in range(num_samples):
        # obtain num_samples inverse hvps
        cur_estimate = v
        
        for depth in range(recursion_depth):
            # epoch-scale recursion depth
            t1 = time.time()
            for img, lb in dataloader:
                img = img.numpy(); lb = lb.numpy()
                x_feed = {model.X: img, model.y:lb}
                hvp = HVP(y,x_feed,cur_estimate)
                # cur_estimate = v + (1-damping)*cur_estimate + 1/scale*(hvp/batch_size)
                cur_estimate = {ks: v[ks] + (1-damping/scale)*cur_estimate[ks] - (1/scale)*hvp[ks]/batch_size for ks in cur_estimate.keys()}
            if verbose:
                print('#w: \n', list(map(lambda x: x.value, params)), '\n#hvp: \n', hvp, '\n#ihvp: \n', cur_estimate)
                print("Recursion depth: {}, norm: {}, time: {} \n".format(depth, np.sqrt(grad_inner_product(cur_estimate,cur_estimate)),time.time()-t1))
        
        inv_hvp = {ks: (1/scale)*cur_estimate[ks] for ks in cur_estimate.keys()}
        inv_hvps.append(inv_hvp)
    
    inv_hvp_val = {ks: np.mean([inv_hvps[i][ks] for i in range(num_samples)], axis=0) for ks in inv_hvps[0].keys()}
    
    return inv_hvp_val

In [13]:
# do this w.r.t. damping
# graphical analysis of stochastic estimation 

def plot_inverse_hvp_se(model, y, v, data_set, **kwargs):
    # Calculate inverse hessian vector product over the training set
    # AND plot the process of convergence w.r.t. 2D space
    # model: neural network model (e.g. model)
    # y: scalar function output of the neural network (e.g. model.loss)
    # v: vector to be producted by inverse hessian (i.e.H^-1 v) (e.g. v_test)
    # data_set: training set to be summed in Hessian
    # kwargs: hyperparameters for stochastic estimation
    recursion_depth = kwargs.pop('recursion_depth', 100) # epoch
    scale = kwargs.pop('scale', 1e1) # similar to learning rate
    damping = kwargs.pop('damping', 0.0) # paper reference: 0.01
    batch_size = kwargs.pop('batch_size', 1)
    num_samples = kwargs.pop('num_samples', 1) # the number of samples(:stochatic estimation of IF) to be averaged
    verbose = kwargs.pop('verbose', False)
    graphics = kwargs.pop('graphics', True)
    
    dataloader = DataLoader(data_set, batch_size, shuffle=True, num_workers=6)
    
    inv_hvps = []
    
    pts = [] # graphical points
    # pts.append(dic2vec(v))
    
    params = y.parameters
    
    for i in range(num_samples):
        # obtain num_samples inverse hvps
        cur_estimate = v
        
        for depth in range(recursion_depth):
            # epoch-scale recursion depth
            t1 = time.time()
            for img, lb in dataloader:
                img = img.numpy(); lb = lb.numpy()
                x_feed = {model.X: img, model.y:lb}
                hvp = HVP(y,x_feed,cur_estimate)
                # cur_estimate = v + (1-damping)*cur_estimate + 1/scale*(hvp/batch_size)
                cur_estimate = {ks: v[ks] + (1-damping/scale)*cur_estimate[ks] - (1/scale)*hvp[ks]/batch_size for ks in cur_estimate.keys()}
            if verbose:
                print('#w: \n', list(map(lambda x: x.value, params)), '\n#hvp: \n', hvp, '\n#ihvp: \n', cur_estimate)
                print("Recursion depth: {}, norm: {}, time: {} \n".format(depth, np.sqrt(grad_inner_product(cur_estimate,cur_estimate)),time.time()-t1))
            if graphics:
                curv = dic2vec(cur_estimate)/scale
                if np.isnan(curv).any() or (np.abs(curv)>1e1).any():
                    pts.append(1e1*np.ones_like(curv))
                else:
                    pts.append(curv)

        inv_hvp = {ks: (1/scale)*cur_estimate[ks] for ks in cur_estimate.keys()}
        inv_hvps.append(inv_hvp)
    
    inv_hvp_val = {ks: np.mean([inv_hvps[i][ks] for i in range(num_samples)], axis=0) for ks in inv_hvps[0].keys()}
    
    return inv_hvp_val, pts

# toy example for IHVP: 1D example

class SimpleNet(object):
    def __init__(self):
        self.X = C.input_variable(shape=(1,))
        self.h = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.X)
        self.pred = C.layers.Dense(1, activation=None, init=C.uniform(1), bias=False)(self.h)
        self.y = C.input_variable(shape=(1,))
        self.loss = C.squared_error(self.pred, self.y)
        
class SimpleDataset(object):
    def __init__(self, images, labels):
        self._images, self._labels = images, labels
    
    def __getitem__(self, index):
        X = self._images[index]
        y = self._labels[index]
        
        return X, y
    
    def __len__(self):
        return len(self._images)


net = SimpleNet()

params = net.pred.parameters

x_feed = {net.X:np.array([[2.]],dtype=np.float32), net.y:np.array([[1.]],dtype=np.float32)}
v_feed = {p: np.ones_like(p.value) for p in params}

print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))
params[0].value = np.asarray([[1.]])
params[1].value = np.asarray([[1./3.]])
print('w1 = \n', params[0].value, '\nw2 = \n', params[1].value, '\nloss = \n', net.loss.eval(x_feed))

print('hvp', HVP(net.loss, x_feed, v_feed))

#images = np.asarray([[2.],[2.]], dtype=np.float32)
#labels = np.asarray([[1.],[1.]], dtype=np.float32)
images = np.asarray([[2.]], dtype=np.float32)
labels = np.asarray([[1.]], dtype=np.float32)

train_set = SimpleDataset(images,labels)
tmp, pts_damp00 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':0.0})
print('damp00:', tmp)
tmp, pts_damp01 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':0.1})
print('damp01:', tmp)
tmp, pts_damp05 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':0.5})
print('damp05:', tmp)
tmp, pts_damp10 = plot_inverse_hvp_se(net, net.loss, v_feed, train_set, **{'damping':1.0})
print('damp10:', tmp)

w1 = 
 [[ 0.0370382]] 
w2 = 
 [[-0.03171763]] 
loss = 
 [ 1.00470448]
w1 = 
 [[ 1.]] 
w2 = 
 [[ 0.33333334]] 
loss = 
 [ 0.1111111]
hvp {Parameter('W', [], [1 x 1]): array([[ 2.22302079]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 9.33413506]], dtype=float32)}
damp00: {Parameter('W', [], [1 x 1]): array([[ 1.24517536]], dtype=float32), Parameter('W', [], [1 x 1]): array([[-0.08125092]], dtype=float32)}
damp01: {Parameter('W', [], [1 x 1]): array([[ 1.08393502]], dtype=float32), Parameter('W', [], [1 x 1]): array([[-0.05433164]], dtype=float32)}
damp05: {Parameter('W', [], [1 x 1]): array([[ 0.71470672]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 0.00550994]], dtype=float32)}
damp10: {Parameter('W', [], [1 x 1]): array([[ 0.50368214]], dtype=float32), Parameter('W', [], [1 x 1]): array([[ 0.03640944]], dtype=float32)}


## Conclusion

- cntk의 구현 상 이유로 automatic differentiation 대신 numerical differentiation을 사용해서 HVP를 구현함.
- Conjugate gradient
    - scipy를 사용해서 구현했고, 검증을 함.
- Stocastic estimation
    - 논문에는 없지만 원저자 코드에 있는 term, minibatch, scale을 추가함.
        - scale이 learning rate와 비슷한 개념임을 예측하고 확인함.
    - 원저자와 달리 damping을 scale로 나눠서 사용해, scale을 바꾸더라도 최종 output이 변화가 없게 함.
    - locally convex한 상황에서 scale을 잘 정한다면 원하던 값에 수렴한다는 것을 확인함.
    - 반대로 말하자면 hyperparameter를 잘 잡아주지 못하면 원하던 값에 수렴시킬 수 없음.